# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 11:36AM,242437,12,HH6451,Hush Hush,Released
1,Jul 21 2022 11:33AM,242436,19,60017988,"GUSA Granules USA, Inc.",Released
2,Jul 21 2022 11:32AM,242435,16,SHL-8300401AJ,"SHL Pharma, LLC",Executing
3,Jul 21 2022 11:30AM,242434,10,PRF-33779,Bio-PRF,Released
4,Jul 21 2022 11:22AM,242433,10,PRF-33729,Bio-PRF,Released
5,Jul 21 2022 11:22AM,242433,10,PRF-33734,Bio-PRF,Released
6,Jul 21 2022 11:22AM,242433,10,PRF-33737,Bio-PRF,Released
7,Jul 21 2022 11:22AM,242433,10,PRF-33740,Bio-PRF,Released
8,Jul 21 2022 11:22AM,242433,10,PRF-33746,Bio-PRF,Released
9,Jul 21 2022 11:22AM,242433,10,PRF-33749,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,242433,Released,10
41,242434,Released,1
42,242435,Executing,1
43,242436,Released,1
44,242437,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242433,NaN,NaN,10.0
242434,NaN,NaN,1.0
242435,NaN,1.0,NaN
242436,NaN,NaN,1.0
242437,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242325,0.0,0.0,1.0
242328,0.0,0.0,1.0
242336,0.0,0.0,1.0
242340,0.0,0.0,1.0
242348,0.0,5.0,73.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242325,0,0,1
242328,0,0,1
242336,0,0,1
242340,0,0,1
242348,0,5,73


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242325,0,0,1
1,242328,0,0,1
2,242336,0,0,1
3,242340,0,0,1
4,242348,0,5,73


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242325,,,1
1,242328,,,1
2,242336,,,1
3,242340,,,1
4,242348,,5,73


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 11:36AM,242437,12,Hush Hush
1,Jul 21 2022 11:33AM,242436,19,"GUSA Granules USA, Inc."
2,Jul 21 2022 11:32AM,242435,16,"SHL Pharma, LLC"
3,Jul 21 2022 11:30AM,242434,10,Bio-PRF
4,Jul 21 2022 11:22AM,242433,10,Bio-PRF
14,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc."
29,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc."
30,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc."
42,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC"
45,Jul 21 2022 10:56AM,242424,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 11:36AM,242437,12,Hush Hush,,,1
1,Jul 21 2022 11:33AM,242436,19,"GUSA Granules USA, Inc.",,,1
2,Jul 21 2022 11:32AM,242435,16,"SHL Pharma, LLC",,1,
3,Jul 21 2022 11:30AM,242434,10,Bio-PRF,,,1
4,Jul 21 2022 11:22AM,242433,10,Bio-PRF,,,10
5,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",,14,1
6,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",,,1
7,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",,,12
8,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",,,3
9,Jul 21 2022 10:56AM,242424,15,"Mizner Bioscience, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:36AM,242437,12,Hush Hush,1,,
1,Jul 21 2022 11:33AM,242436,19,"GUSA Granules USA, Inc.",1,,
2,Jul 21 2022 11:32AM,242435,16,"SHL Pharma, LLC",,1,
3,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1,,
4,Jul 21 2022 11:22AM,242433,10,Bio-PRF,10,,
5,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",1,14,
6,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",1,,
7,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",12,,
8,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",3,,
9,Jul 21 2022 10:56AM,242424,15,"Mizner Bioscience, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:36AM,242437,12,Hush Hush,1,,
1,Jul 21 2022 11:33AM,242436,19,"GUSA Granules USA, Inc.",1,,
2,Jul 21 2022 11:32AM,242435,16,"SHL Pharma, LLC",,1,
3,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1,,
4,Jul 21 2022 11:22AM,242433,10,Bio-PRF,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:36AM,242437,12,Hush Hush,1.0,NaN,NaN
1,Jul 21 2022 11:33AM,242436,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Jul 21 2022 11:32AM,242435,16,"SHL Pharma, LLC",NaN,1.0,NaN
3,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1.0,NaN,NaN
4,Jul 21 2022 11:22AM,242433,10,Bio-PRF,10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:36AM,242437,12,Hush Hush,1.0,0.0,0.0
1,Jul 21 2022 11:33AM,242436,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Jul 21 2022 11:32AM,242435,16,"SHL Pharma, LLC",0.0,1.0,0.0
3,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1.0,0.0,0.0
4,Jul 21 2022 11:22AM,242433,10,Bio-PRF,10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2908781,143.0,19.0,5.0
12,484773,2.0,0.0,0.0
15,969700,18.0,14.0,0.0
16,1454467,8.0,1.0,0.0
19,1454362,17.0,2.0,0.0
20,484761,76.0,5.0,0.0
21,1939029,8.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2908781,143.0,19.0,5.0
1,12,484773,2.0,0.0,0.0
2,15,969700,18.0,14.0,0.0
3,16,1454467,8.0,1.0,0.0
4,19,1454362,17.0,2.0,0.0
5,20,484761,76.0,5.0,0.0
6,21,1939029,8.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,143.0,19.0,5.0
1,12,2.0,0.0,0.0
2,15,18.0,14.0,0.0
3,16,8.0,1.0,0.0
4,19,17.0,2.0,0.0
5,20,76.0,5.0,0.0
6,21,8.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,143.0
1,12,Released,2.0
2,15,Released,18.0
3,16,Released,8.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,19.0,0.0,14.0,1.0,2.0,5.0,0.0
Released,143.0,2.0,18.0,8.0,17.0,76.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,14.0,1.0,2.0,5.0,0.0
2,Released,143.0,2.0,18.0,8.0,17.0,76.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,14.0,1.0,2.0,5.0,0.0
2,Released,143.0,2.0,18.0,8.0,17.0,76.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()